# Happy Customers Project

## Data Description:

Y = target attribute (Y) with values indicating 0 (unhappy) and 1 (happy) customers <br/>
X1 = my order was delivered on time <br/>
X2 = contents of my order was as I expected <br/>
X3 = I ordered everything I wanted to order <br/>
X4 = I paid a good price for my order <br/>
X5 = I am satisfied with my courier <br/>
X6 = the app makes ordering easy for me <br/>

Attributes X1 to X6 indicate the responses for each question and have values from 1 to 5 where the smaller number indicates less and the higher number indicates more towards the answer.

### Load Libraries

In [1]:
# useful libraries
import pandas as pd
import numpy as np

# EDA
from pandas_profiling import ProfileReport

# feature selection
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

# train test split
from sklearn.model_selection import train_test_split

# model, hyperparameter search and scoring
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import roc_auc_score

c:\Users\nntor\anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


### Load Dataset and EDA

In [2]:
# load dataset
data = pd.read_csv("Data/ACME-HappinessSurvey2020.csv")
print(data)

# perform EDA on the whole dataset to understand the data 
profile = ProfileReport(data)
profile.to_widgets()

     Y  X1  X2  X3  X4  X5  X6
0    0   3   3   3   4   2   4
1    0   3   2   3   5   4   3
2    1   5   3   3   3   3   5
3    0   5   4   3   3   3   5
4    0   5   4   3   3   3   5
..  ..  ..  ..  ..  ..  ..  ..
121  1   5   2   3   4   4   3
122  1   5   2   3   4   2   5
123  1   5   3   3   4   4   5
124  0   4   3   3   4   4   5
125  0   5   3   2   5   5   5

[126 rows x 7 columns]


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

### Feature Selection

In [3]:
# create two dataset: one with the target variable and one with the features
Y = data.iloc[:, 0]
X = data.drop('Y', axis=1)

# define feature selection
k = 4
fs = SelectKBest(score_func=f_classif, k=k)

# apply feature selection
X_selected = fs.fit_transform(X, Y)
X_selected = pd.DataFrame(X_selected)

# print the pvalue of the features and the selected features
pvalues = pd.DataFrame(fs.pvalues_, list(X.columns))
pvalues.columns = ['p-value']
print(pvalues)

selectedFeatures = list(X.columns[ sorted(np.argsort(fs.pvalues_)[:k]) ])
print(f'\nTop {k} Selected Features: {selectedFeatures}') 

# take the indices of the selected features with: sorted(np.argsort(fs.pvalues_)[:-(k - 2)])

     p-value
X1  0.001486
X2  0.787313
X3  0.091807
X4  0.473623
X5  0.011488
X6  0.060568

Top 4 Selected Features: ['X1', 'X3', 'X5', 'X6']


### Hyperparameter tuning of our model

In [49]:
# split the dataset into train test
X_train, X_test, y_train, y_test = train_test_split(X[selectedFeatures], Y, test_size=0.33, random_state=25)

# 
def my_roc_auc_score(model, X, y): return roc_auc_score(y, model.predict_proba(X)[:,1])

# search space for our parameters

params = { 'learning_rate': [0.001, 0.01, 0.05, 0.1, 0.2],
           'subsample': np.arange(0.2, 0.5, 0.02),
           'min_child_weight': np.arange(1, 20, 2),
           'gamma': np.arange(1, 20, 2),
           'max_depth': np.arange(5, 30, 5),
           'colsample_bytree': np.arange(0.1, 0.5, 0.02),
           'n_estimators': [100, 200, 300, 400, 600]}

# create model
xgbr = XGBClassifier(use_label_encoder = False,
                     objective= 'binary:logistic',
                     tree_method = 'hist',
                     eval_metric = 'auc')

# create randomized search
clf = RandomizedSearchCV(estimator=xgbr,
                         param_distributions=params,
                         scoring = my_roc_auc_score,
                         n_iter=80,
                         verbose=1)

# fit model to find best hyperparameters
clf.fit(X_train, y_train)

Fitting 5 folds for each of 80 candidates, totalling 400 fits


c:\Users\nntor\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
c:\Users\nntor\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
c:\Users\nntor\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
c:\Users\nntor\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future

RandomizedSearchCV(estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           enable_categorical=False,
                                           eval_metric='auc', gamma=None,
                                           gpu_id=None, importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate=None,
                                           max_delta_step=None, max_depth=None,
                                           min_child_weight=None, missing=nan,
                                           monoton...
                                        'gamma': array([ 1,  3,  5,  7,  9, 11, 13, 15, 17, 19]),
                                        'learning_rate': [0

### Results of hyperparameter tuning

In [50]:
df_cv_results = pd.DataFrame(clf.cv_results_)
df_cv_results = df_cv_results[["rank_test_score","mean_test_score",
                               "param_learning_rate", 
                               "param_max_depth", 
                               "param_n_estimators", 
                               "param_subsample", 
                               "param_colsample_bytree", 
                               "param_min_child_weight", 
                               "param_gamma"]]
df_cv_results.sort_values(by='rank_test_score', inplace=True)
df_cv_results[:10]

,rank_test_score,mean_test_score,param_learning_rate,param_max_depth,param_n_estimators,param_subsample,param_colsample_bytree,param_min_child_weight,param_gamma
22,1,0.706429,0.001,25,600,0.38,0.32,1,9
10,1,0.706429,0.001,5,600,0.38,0.36,1,9
43,3,0.700952,0.2,10,100,0.26,0.26,1,3
24,4,0.696111,0.001,5,600,0.4,0.36,1,3
69,5,0.685556,0.1,10,300,0.3,0.36,1,1
70,6,0.675317,0.001,10,200,0.38,0.38,3,5
31,7,0.670992,0.001,20,100,0.4,0.44,1,5
47,8,0.652262,0.01,15,600,0.26,0.2,3,3
51,9,0.636468,0.1,10,300,0.32,0.24,1,9
15,10,0.592619,0.1,10,600,0.4,0.26,1,13


### Final Model

In [53]:
model_xgboost_fin = XGBClassifier(objective= 'binary:logistic',

                                  learning_rate=0.001,
                                  max_depth=25,
                                  n_estimators=600,
                                  subsample=0.38,
                                  colsample_bytree=0.32,
                                  min_child_weight=1,
                                  gamma = 2,

                                  eval_metric='auc',
                                  tree_method = 'hist',
                                  verbosity=1,
                                  use_label_encoder=False)



model_xgboost_fin.fit(X_train,
                      y_train,
                      verbose=True)

c:\Users\nntor\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.32,
              enable_categorical=False, eval_metric='auc', gamma=2, gpu_id=-1,
              importance_type=None, interaction_constraints='',
              learning_rate=0.001, max_delta_step=0, max_depth=25,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=600, n_jobs=12, num_parallel_tree=1,
              predictor='auto', random_state=0, reg_alpha=0, reg_lambda=1,
              scale_pos_weight=1, subsample=0.38, tree_method='hist',
              use_label_encoder=False, validate_parameters=1, verbosity=1)

### Train and Test scores

In [54]:
y_train_pred = model_xgboost_fin.predict_proba(X_train)[:,1]
y_test_pred = model_xgboost_fin.predict_proba(X_test)[:,1]

print("AUC Train: {:.4f}\nAUC Test: {:.4f}".format(roc_auc_score(y_train, y_train_pred),
                                                    roc_auc_score(y_test, y_test_pred)))

AUC Train: 0.7585
AUC Test: 0.5875


### Feature Importance

In [55]:
df_var_imp = pd.DataFrame({"Variable": selectedFeatures,
                           "Importance": model_xgboost_fin.feature_importances_}) \
                        .sort_values(by='Importance', ascending=False)
df_var_imp

,Variable,Importance
0,X1,0.347202
3,X6,0.226812
2,X5,0.225180
1,X3,0.200806
